In [99]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from rapidfuzz import process

In [63]:
import os
import sys

src_dir = os.path.join(os.getcwd(), "..", "..")
from importlib import reload
sys.path.append(os.path.abspath(src_dir))

import src.helpers.city_to_train_station

reload(src.helpers.city_to_train_station)

from src.helpers.city_to_train_station import VisualizerHelper

In [35]:
train_station_df = pd.read_csv('../data/liste-des-gares.csv', sep=';')
cities_df = pd.read_csv('../data/cities.csv')


city_names_df = cities_df[["label", "latitude", "longitude", "region_name", "department_name"]].copy()
city_names_df.rename(columns={"region_name": "region", "department_name": "department"}, inplace=True)
city_names_df["status"] = "city"

train_station_names_df = pd.DataFrame(train_station_df["LIBELLE"]).copy()
train_station_names_df.rename(columns={"LIBELLE": "label"}, inplace=True)
train_station_names_df[["latitude", "longitude"]] = train_station_df["C_GEO"].str.split(',', expand=True).astype(float)
train_station_names_df["status"] = "train_station"

In [293]:
all_names_df = pd.concat([city_names_df, train_station_names_df], ignore_index=True)
all_names_df["label"] = all_names_df["label"].str.lower()
all_names_df["label"] = all_names_df["label"].str.replace("-", " ")
all_names_df["region"] = all_names_df["region"].str.lower()
all_names_df["region"] = all_names_df["region"].str.replace("-", " ")
all_names_df["department"] = all_names_df["department"].str.lower()
all_names_df["department"] = all_names_df["department"].str.replace("-", " ")
all_names_df.loc[all_names_df["status"] == "train_station", ["region", "department"]] = "-"

In [100]:
helper = VisualizerHelper()

with tqdm(total=all_names_df.loc[all_names_df["status"] == "city"].shape[0], unit="name") as pbar:
    for row in all_names_df.loc[all_names_df["status"] == "city"].iterrows():
        nearest_train_station = helper.get_nearest_station(row[1]["label"]).LIBELLE.values[0]
        all_names_df.loc[all_names_df["label"] == row[1]["label"], "nearest_train_station"] = nearest_train_station
        
        pbar.update(1)

  0%|          | 0/38932 [00:00<?, ?name/s]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


In [105]:
all_names_df = all_names_df.drop(
    all_names_df.loc[
        (all_names_df["status"] == "city") & (all_names_df["nearest_train_station"].isna())
    ].index
)
all_names_df.loc[all_names_df["status"] == "train_station", "nearest_train_station"] = "-"


In [250]:
def fuzzy_search(name):
    name = name.lower().replace("-", " ")
    result_df = []

    result_label = process.extractOne(name, all_names_df["label"].values)
    result_region = process.extractOne(name, all_names_df["region"].values)
    result_department = process.extractOne(name, all_names_df["department"].values)

    result_df.append(
        {"name": result_label[0], "score": result_label[1], "status": "city"}
    )

    result_df.append(
        {"name": result_region[0], "score": result_region[1], "status": "region"}
    )
    
    result_df.append(
        {"name": result_department[0], "score": result_department[1], "status": "department"}
    )

    result_df = pd.DataFrame(result_df)

    if not result_df.empty:
        best_result = result_df.loc[result_df["score"].idxmax()]
        return best_result
    else:
        return None

In [303]:
fuzzy_search("Bretagne")

       name  score      status
0  bretagne  100.0        city
1  bretagne  100.0      region
2      tarn   67.5  department


name      bretagne
score        100.0
status        city
Name: 0, dtype: object

In [167]:
all_names_df.to_csv("../data/cities_and_train_stations.csv", index=False)